In [ ]:
import netifaces as ni
from scapy.all import ARP, Ether, srp
from ipaddress import ip_network
import subprocess


def get_network_range(interface='eth0'):
    """
    Get the network range for the given interface.
    """
    
    
    # Get the IP address and netmask of the default interface
    addr = ni.ifaddresses(interface)[ni.AF_INET][0]['addr']
    netmask = ni.ifaddresses(interface)[ni.AF_INET][0]['netmask']

    # Calculate network range
    network = ip_network(f"{addr}/{netmask}", strict=False)
    return str(network)

def scan_network(ip_range):
    """
    Scan the network to find devices.
    """
    
    
    # Create an ARP request packet
    arp_request = ARP(pdst=ip_range)
    broadcast = Ether(dst="ff:ff:ff:ff:ff:ff")
    arp_request_broadcast = broadcast/arp_request

    # Send the packet and receive responses
    answered_list = srp(arp_request_broadcast, timeout=1, verbose=False)[0]

    # List of devices
    devices = []

    for sent, received in answered_list:
        # For each response, add IP and MAC address to devices list
        devices.append({'ip': received.psrc, 'mac': received.hwsrc})

    return devices

# Attempt to automatically select a network interface
default_interface = ni.gateways()['default'][ni.AF_INET][1]
network_range = get_network_range(default_interface)

devices = scan_network(network_range)

device_string = ""

for device in devices:
    device_string += f"IP Address: {device['ip']}, MAC Address: {device['mac']}\n"

# Define the binary path
binary_path = './myBinary'

# Start the process
process = subprocess.Popen([binary_path], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Send the input and get the output
stdout, stderr = process.communicate(input=device_string)